# Tutorial for FHDpy

Credits: Henrique Ennes (https://hlovisiennes.github.io/).

In [1]:
import FHDpy.FHD as FHD
from FHDpy.SLP import SLP as SLP

This notebook deals with the data structure for Heegaard diagrams described in [arxiv preprint](https://arxiv.org/abs/2507.11406). We strongly advise the reader to first check `TutorialSLP.ipynb`, as many methods of that module will be implicitly or explicitly used here.

## Introduction

The main idea of `FHDpy` is to give us a data structure to represent closed 3-manifolds that is significantly more compact (in fact, sometimes exponentially) than the usual representation of 3-manifolds using **triangulations**.  For such, we will use Heegaard splittings. In a snipshot, a **Heegaard splitting** is given by a pair $(\Sigma_g, \phi)$, where $\Sigma_g$ is the (unique up to homeomorphism) closed surface of genus $g$ and $\phi$ is a homeomorphism from $\Sigma_g\to\Sigma_g$, often restrcited to the mapping class group of $\Sigma_g$, $\text{Mod}(\Sigma_g)$, i.e., considered only up to isotopies. One can imagine obtaining the 3-manifold by gluing two handlebodies of boundary $\Sigma_g$ using $\phi$ as a gluing map. In particular, we can always assume that $\phi$ is a word in some finite alphabet consisting of Dehn twists about Lickorish generators and can be thus described as a data structure.

The `Twister` module of `SnapPy` allows for getting a triangulation out of a Heegaard splitting of a 3-manifold. For example, it is a well-known fact that the 3-sphere $S^3$ has a Heegaard splitting for $g=1$ (i.e., $\Sigma_g$ is the torus) with $\phi = b$ where $b$ represents the Dehn twist about one of the two Lickorish generators of the torus, in this case, the meridian of the torus (the purple curve in the image below).

In [15]:
torus = snappy.twister.Surface('S_1')
gluing = 'b'
manifold = torus.splitting(gluing= gluing, handles= 'a*A')

We can check that this, indeed, represents a sphere by computing its fundamental group (which, by the Poincaré conjecture, proved by Perelman, is the only 3-manifold with trivial fundamental group).

In [16]:
manifold.fundamental_group()

Generators:
   
Relators:
   

The topological information of a Heegaard splitting can be also given as **Heegaard diagram**, consisting of a tuple $(\Sigma_g, \alpha, \beta)$ where $\alpha$ and $\beta$ are two curves in $\Sigma_g$ surface such that $\phi(\alpha)=\beta$. In terms of data structure, we assume that

1. $\Sigma_g$ is given by a 2-complex $T$.
2. $\alpha$ are $g$ curves in the edges of $T^{(1)}$ which may only intersect at vertices. We call the $\alpha$ curves edged (as they lie in the edges of the complex).
3. $\beta$ are $g$ SLPs describing the signed intersection sequences with each of the eges of $T$. We call the $\beta$ curves edged (as they lie in the edges of the complex).

For example, the following is a diagram of $S^3$ over the torus (in fact, it is a diagram of the same spltting described above). Here the unique $\alpha$ curve is shown in red, the unique $\beta$ curve in blue, and the edges of the complex (here a square) representing the torus are all labeled in black.

## Diagrams over the torus
In FHD, we can represent this same diagram in two ways, `FHD_genus1` and `FHDLong`. As the name indicates, although `FHDLong` can be used to find Heegaard diagrams of 3-manifolds over general surfaces, `FHD_genus1` can be used only to represent 3-manifolds that have splittings over the torus (also known as lens spaces). Starting with the more general one, `FHDLong`, it allows one to express the cellular complex for $\Sigma_g=T^2$. This syntax assumes four mandatory arguments and one optional. The mandatory are:

1. The $\alpha$ curves in standard form, that is, slightly displaced from the edges of the complex. In the current version, these need to be computed by hand from the complex. We assume that they are given as SLPs. Each of the curves is assumed to be a key in a dictionary whose values are the SLP of the intersection sequence.

2. The $\alpha$ curves as edges of the complex. Each of the curves is assumed to be a key in a dictionary whose values are the list of edges.

3. The $\alpha$ curves in standard form, that is, slightly displaced from the edges of the complex. In the current version, these need to be computed by hand from the complex. We assume that they are given as SLPs. Each of the curves is assumed to be a key in a dictionary whose values are the SLP of the intersection sequence.

4. The Lickorish generators as edges of the complex. Each of the curves is assumed to be a key in a dictionary whose values are the list of edges.

and finally, the optional $\beta$ curves, which are necessarily standard. Again, we assume that they are given as SLPs.

In [2]:
import importlib

importlib.reload(FHD)

alpha_curves_standard = {'a': SLP(['b*'])}
alpha_curves_edges = {'a': ['a']}
generators_standard = {'a': SLP(['b']), 'b': SLP(['a*'])}
generators_edge = {'a':['a'], 'b':['b']}
beta = [SLP(['a'])]

# This creates the diagram itself.
fhd_compressed = FHD.FHDLong(alpha_curves_standard, alpha_curves_edges, generators_standard, generators_edge, beta = beta)

Again, we can check if this is a sphere by computing its fundamental group. 

In [10]:
print('Generators: ', list(fhd_compressed.alpha.keys()))
print('Relators: ', fhd_compressed.fundamental_group()[0].get_uncompressed().list_form)

Generators:  ['a']
Relators:  ['a']


The syntax of `FHD_genus1` is significantly simpler, but also more restrictive.

## Diagrams over the double torus